<a href="https://colab.research.google.com/github/lucaslopes/partos/blob/csv/Vari%C3%A1veis_API_PCDAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Código

## Bibliotecas

In [1]:
import os
import gspread
import requests
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import gspread_dataframe as gd

from tqdm import tqdm
from getpass import getpass
from google.colab import auth
from bs4 import BeautifulSoup
from google.auth import default
from plotly.subplots import make_subplots

## Google Spreadsheet

In [2]:
def read_df_gpread(spreadsheet, ws_name):
  worksheet = spreadsheet.worksheet(ws_name) # read table
  rows = worksheet.get_all_values() # get_all_values gives a list of rows.
  df = pd.DataFrame.from_records(rows) # # Convert to a DataFrame and render.
  new_header = df.iloc[0] #grab the first row for the header
  df = df[1:] #take the data less the header row
  df.columns = new_header #set the header row as the df header
  return df

In [3]:
def insert_df_gspread(df, spreadsheet, ws_name):
  try:
    ws = spreadsheet.add_worksheet(
      title=ws_name, rows=df.shape[0], cols=df.shape[1])
  except:
    ws = spreadsheet.worksheet(ws_name)
  return gd.set_with_dataframe(ws, df)

In [4]:
def get_dict_vars(url):
  res = requests.get(url)
  soup = BeautifulSoup(res.content)
  table = soup.find_all('table')
  df = pd.read_html(
    str(table)
  )[1].rename(columns={0:'Variável', 1: 'Tipo', 2: 'Descrição'})
  return df

In [5]:
def table_from_url_to_gspread(url, label):
  df = get_dict_vars(url)
  insert_df_gspread(df, f'{label}_raw')

In [6]:
def urls_to_gpread(urls, labels):
  for url, label in zip(urls, labels):
    table_from_url_to_gspread(url, label)

## Requisições SQL à API

In [7]:
def get_dict_vars(url):
  res = requests.get(url)
  soup = BeautifulSoup(res.content)
  table = soup.find_all('table')
  df = pd.read_html(
    str(table)
  )[1].rename(columns={0:'Variável', 1: 'Tipo', 2: 'Descrição'})
  return df

In [8]:
def compose_request(query, token):
	return {
		"token": {"token": token},
		"sql": {"sql": {"query": query}}}

In [9]:
def sql_query(query, url, token):
	return requests.post(
		os.path.join(
			url,'sql_query'),
			json = compose_request(query, token))

In [10]:
def json_to_df(resp):
	resp = resp.json()
	if 'RequestError' in resp or 'columns' not in resp:
		df = pd.DataFrame()
		print(resp)
	else:
		columns = [col['name'] for col in resp['columns']]
		values = resp['rows']
		df = pd.DataFrame(values, columns=columns)
	return df

In [11]:
def df_query(query, url, token):
	return json_to_df(sql_query(query, url, token))

## SQL

In [12]:
def count_var(var, database="datasus-sinasc"):
  return f'select null as label, count({var}) as records from {database}'

In [13]:
def count_nan(var, database="datasus-sinasc"):
  return f'select null as label, count(*) - count({var}) as records from {database}'

In [14]:
def groupby_var(var, database="datasus-sinasc"):
  return f'select {var} as label, count({var}) as records from {database} group by {var} order by {var} asc'

In [15]:
def get_database_with_filters(database, filters='True'):
  return f'"datasus-{database}" where {filters}'

In [16]:
def merge_filters(filters):
  fs = ' and '.join([
    f'({filter})' for filter in filters])
  return fs if len(fs) > 0 else 'True'

## Investigação de Variáveis

In [17]:
def var_records(var, db, filters, url_api, token):
  database = get_database_with_filters(db, filters)
  df_count = df_query(count_var(var, database), url_api, token)
  df_count['label'] = df_count['label'].fillna(var)
  df_nan = df_query(count_nan(var, database), url_api, token).fillna('NaN')
  df_group = df_query(groupby_var(var, database), url_api, token)
  df_count['variavel'] = var
  df_nan['variavel'] = var
  df_group['variavel'] = var
  df_count['type'] = 'count'
  df_nan['type'] = 'count'
  df_group['type'] = 'group'
  return pd.concat(
    [df_count, df_nan, df_group],
    ignore_index=True,
  )[['variavel', 'type', 'label', 'records']]

In [18]:
def consult_all_vars(db, filters, vars, url_api, token):
  dfs = list()
  for var in tqdm(vars):
    try:
      dfs.append(var_records(var, db, filters, url_api, token))
    except Exception as E:
      print(var, E)
  return pd.concat(dfs, ignore_index=True)

In [19]:
def consult_all_databases(databases, url_api, token):
  dfs = list()
  for db, infos in databases.items():
    df = consult_all_vars(db, infos['filters'], infos['vars'], url_api, token)
    df['database'] = db
    dfs.append(df)
  return pd.concat(dfs, ignore_index=True)[
    ['database', 'variavel', 'type', 'label', 'records']]

In [20]:
def get_vars_from_df(df):
  return df[df['Status'] != 'Descartar']['Variável'].values

## Visualização dos Gráficos

In [21]:
def pie_vars_count(df, label='label', recs='records'):
  return go.Pie(
    labels=df[label],
    values=df[recs],
  )

In [22]:
def bar_vars_count(df, label='label', recs='records'):
  return go.Bar(
    x=df[label],
    y=df[recs],
  )

In [23]:
def fig_pie_bar(df, count_desc=False):

  fig = make_subplots(rows=1, cols=2,
    specs=[[{"type": "domain"}, {"type": "xy"}]])

  df_pie = df[df['type'] == 'count']
  plot_pie = pie_vars_count(df_pie)
  fig.add_trace(plot_pie, row=1, col=1)

  df_bar = df[df['type'] == 'group']
  if count_desc:
    df_bar = df_bar.sort_values(
      'records', ascending=False)
  plot_bar = bar_vars_count(df_bar)
  fig.add_trace(plot_bar, row=1, col=2)

  fig.update_layout(
    height=400, width=800,
    title_text="Left is not NaN count / Right is values count")

  return fig

In [24]:
def plot_db_var(database, var, count_desc=False):
  df = df_counts[(
    (df_counts['database'] == database)
    &
    (df_counts['variavel'] == var)
  )]
  return fig_pie_bar(df, count_desc)

## Tabelas das Variáveis

In [25]:
def get_databases(spreadsheet, db_sih='sih', db_sinasc='sinasc'):
  sih = read_df_gpread(spreadsheet, db_sih)
  sinasc = read_df_gpread(spreadsheet, db_sinasc)
  return  {
    db_sih : {
      'filters' : merge_filters(filters_sih),
      'vars' : get_vars_from_df(sih)[:1],
    },
    db_sinasc : {
      'filters' : merge_filters(filters_sinasc),
      'vars' : ['def_sexo', 'DTNASC'],#get_vars_from_df(sinasc),
    },
  }

# Execução

## Configuração

In [26]:
skip_authenticate = True # True | False
skip_api = skip_authenticate
skip_google = skip_authenticate

In [27]:
gspread_name = 'Variáveis PCDaS'
df_counts_name = 'dbs_count_f_2010_19' # dbs_count_f_2010_19 | dbs_count_no_filters

## Definição Filtro 100%

In [28]:
filters_sih = [
  # "def_procedimento_realizado like 'PARTO%'",
  "PROC_REA in ('0310010039', '0411010034')",
  "ano_internacao between 2010 and 2019",
]

In [29]:
filters_sinasc = [
  "ano_nasc between 2010 and 2019",
]

## URLS

In [30]:
URL_API = 'https://bigdata-api.fiocruz.br'

In [31]:
URL_DICT_SIH = 'https://pcdas.icict.fiocruz.br/conjunto-de-dados/sistema-de-informacoes-hospitalares-do-sus-sihsus/dicionario-de-variaveis/'

In [32]:
URL_DICT_SINASC = 'https://pcdas.icict.fiocruz.br/conjunto-de-dados/sistema-de-informacao-sobre-nascidos-vivos/dicionario-de-variaveis/'

In [33]:
URL_DICTS = [URL_DICT_SIH, URL_DICT_SINASC]

In [34]:
URL_COUNT_F_2010_19 = 'https://raw.githubusercontent.com/lucaslopes/partos/csv/data/dbs_count_f_2010_19.csv'

In [35]:
URL_COUNT_NO_FILTERS = 'https://raw.githubusercontent.com/lucaslopes/partos/csv/data/dbs_count_no_filters.csv'

## API PCDaS

In [36]:
if not skip_api:
  API_KEY = getpass('Enter the API KEY:')

## Autenticação Google

In [37]:
if not skip_google:
  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds)
  gs_sheet = gc.open(gspread_name)

In [38]:
# urls_to_gpread(URL_DICTS, ['sih', 'sinasc'])

## Carregando Dados

In [39]:
if not skip_api and not skip_google:
  databases = get_databases(gs_sheet)
  df_counts = consult_all_databases(databases, URL_API, API_KEY)
  # insert_df_gspread(df_counts, gs_sheet, df_counts_name)
  df_counts.to_csv(f'{df_counts_name}.csv', index=False)

In [40]:
looking_scope = URL_COUNT_F_2010_19 # URL_COUNT_F_2010_19 | URL_COUNT_F_2010_19
# df_counts = read_df_gpread(gs_sheet, df_counts_name)
# df_counts = pd.read_csv(f'{df_counts_name}.csv')
df_counts = pd.read_csv(looking_scope)
df_counts

,database,variavel,type,label,records
0,sih,ANO_CMPT,count,ANO_CMPT,17109256
1,sih,ANO_CMPT,count,NaN,0
2,sih,ANO_CMPT,group,2010,1716548
3,sih,ANO_CMPT,group,2011,1779066
4,sih,ANO_CMPT,group,2012,1706876
...,...,...,...,...,...
25310,sinasc,res_codigo_adotado,group,230700,4079
25311,sinasc,res_codigo_adotado,group,230710,4427
25312,sinasc,res_codigo_adotado,group,230720,1238
25313,sinasc,res_codigo_adotado,group,230725,3159


**SIH**

100%|██████████| 37/37

[06:53<00:00, 11.17s/it]

**SINASC**

100%|██████████| 52/52 

[12:55<00:00, 14.92s/it]

# Variáveis do SINASC

In [41]:
db_sinasc = 'sinasc'
vars_sinasc = list()

## Hospital

### `CODESTAB`

Código de estabelecimento

In [42]:
var = 'CODESTAB'
plot_db_var(db_sinasc, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [43]:
decision = 1

**Possui algum filtro?**

In [44]:
var_filter = ''

**Gostaria de renomear a variável?**

In [45]:
renomear = 'cnes_hosp'

**Registrando decisão**

In [46]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `STDNNOVA`

Status de DO Nova. Valores:
- 1 – SIM;
- 0 – NÃO.

In [47]:
var = 'STDNNOVA'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [48]:
decision = 3

**Possui algum filtro?**

In [49]:
var_filter = ''

**Gostaria de renomear a variável?**

In [50]:
renomear = ''

**Registrando decisão**

In [51]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `STDNEPIDEM`

Status de DO Epidemiológica. Valores:
- 1 – SIM;
- 0 – NÃO.

In [52]:
var = 'STDNEPIDEM'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [53]:
decision = 3

**Possui algum filtro?**

In [54]:
var_filter = ''

**Gostaria de renomear a variável?**

In [55]:
renomear = ''

**Registrando decisão**

In [56]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

## Procedimento

### `ano_nasc`

Ano do nascimento

In [57]:
var = 'ano_nasc'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [58]:
decision = 1

**Possui algum filtro?**

In [59]:
var_filter = 'ano_nasc between 2010 and 2019'

**Gostaria de renomear a variável?**

In [60]:
renomear = 'ano'

**Registrando decisão**

In [61]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `APGAR1`

Apgar no primeiro minuto 00 a 10

In [62]:
var = 'APGAR1'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [63]:
decision = 2

**Possui algum filtro?**

In [64]:
var_filter = 'APGAR1 between 0 and 10'

**Gostaria de renomear a variável?**

In [65]:
renomear = ''

**Registrando decisão**

In [66]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `APGAR5`

Apgar no quinto minuto 00 a 10

In [67]:
var = 'APGAR5'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [68]:
decision = 2

**Possui algum filtro?**

In [69]:
var_filter = 'APGAR5 between 0 and 10'

**Gostaria de renomear a variável?**

In [70]:
renomear = ''

**Registrando decisão**

In [71]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `def_consultas`

Número de consultas durante o pré-natal
- Nenhuma;
- de 1 a 3;
- de 4 a 6;
- 7 e mais;
- Ignorado




In [72]:
var = 'def_consultas'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [73]:
decision = 2

**Possui algum filtro?**

In [74]:
var_filter = 'def_consultas != "Ignorado"'

**Gostaria de renomear a variável?**

In [75]:
renomear = 'pre_natal'

**Registrando decisão**

In [76]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `CONSULTAS`

Número de consultas de pré-natal:
- 1: Nenhuma;
- 2: de 1 a 3;
- 3: de 4 a 6;
- 4: 7 e mais;
- 9: Ignorado

In [77]:
var = 'CONSULTAS'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [78]:
decision = 3

**Possui algum filtro?**

In [79]:
var_filter = ''

**Gostaria de renomear a variável?**

In [80]:
renomear = ''

**Registrando decisão**

In [81]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `CONSPRENAT`

Número de consultas pré‐natal

In [82]:
var = 'CONSPRENAT'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [83]:
decision = 3

**Possui algum filtro?**

In [84]:
var_filter = ''

**Gostaria de renomear a variável?**

In [85]:
renomear = ''

**Registrando decisão**

In [86]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `STTRABPART`

Trabalho de parto induzido? Valores:
- 1 – Sim;
- 2 – Não;
- 3 – Não se aplica;
- 9 – Ignorado.

In [87]:
var = 'STTRABPART'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [88]:
decision = 3

**Possui algum filtro?**

In [89]:
var_filter = ''

**Gostaria de renomear a variável?**

In [90]:
renomear = ''

**Registrando decisão**

In [91]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `TPFUNCRESP`

Tipo de função do responsável pelo preenchimento. Valores:
- 1 – Médico;
- 2 – Enfermeiro;
- 3 – Parteira;
- 4 – Funcionário docartório;
- 5 – Outros.

In [92]:
var = 'TPFUNCRESP'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [93]:
decision = 3

**Possui algum filtro?**

In [94]:
var_filter = ''

**Gostaria de renomear a variável?**

In [95]:
renomear = ''

**Registrando decisão**

In [96]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `TPMETESTIM`

Método utilizado. Valores:
- 1 – Exame físico;
- 2 – Outro método;
- 9 – Ignorado.

In [97]:
var = 'TPMETESTIM'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [98]:
decision = 3

**Possui algum filtro?**

In [99]:
var_filter = ''

**Gostaria de renomear a variável?**

In [100]:
renomear = ''

**Registrando decisão**

In [101]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `TPNASCASSI`

Nascimento foi assistido por? Valores:
- 1 – Médico;
- 2 – Enfermeira/obstetriz;
- 3 – Parteira;
- 4 – Outros;
- 9 – Ignorado.




In [102]:
var = 'TPNASCASSI'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [103]:
decision = 3

**Possui algum filtro?**

In [104]:
var_filter = ''

**Gostaria de renomear a variável?**

In [105]:
renomear = ''

**Registrando decisão**

In [106]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

## Localidade

### `CODMUNNASC`

Município de ocorrência, em codificação idêntica a de CODMUNRES, conforme tabela TABMUN

In [107]:
var = 'CODMUNNASC'
plot_db_var(db_sinasc, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [108]:
decision = 1

**Possui algum filtro?**

In [109]:
var_filter = ''

**Gostaria de renomear a variável?**

In [110]:
renomear = ''

**Registrando decisão**

In [111]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `nasc_codigo_adotado`

Armazena o código atribuído ao município de nascimento atualmente, tratando os casos em que múltiplos códigos tenham sido utilizados para um mesmo município ao longo do tempo

In [112]:
var = 'nasc_codigo_adotado'
plot_db_var(db_sinasc, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [113]:
decision = 1

**Possui algum filtro?**

In [114]:
var_filter = ''

**Gostaria de renomear a variável?**

In [115]:
renomear = ''

**Registrando decisão**

In [116]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `nasc_MUNNOME`

Nome do município de nascimento

In [117]:
var = 'nasc_MUNNOME'
plot_db_var(db_sinasc, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [118]:
decision = 1

**Possui algum filtro?**

In [119]:
var_filter = ''

**Gostaria de renomear a variável?**

In [120]:
renomear = ''

**Registrando decisão**

In [121]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `nasc_MUNNOMEX`

Nome do município de nascimento em maiúsculas e sem acentos

In [122]:
var = 'nasc_MUNNOMEX'
plot_db_var(db_sinasc, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [123]:
decision = 1

**Possui algum filtro?**

In [124]:
var_filter = ''

**Gostaria de renomear a variável?**

In [125]:
renomear = ''

**Registrando decisão**

In [126]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `nasc_RSAUDCOD`

Código da Regional de Saúde a que o Município de nascimento pertence

In [127]:
var = 'nasc_RSAUDCOD'
plot_db_var(db_sinasc, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [128]:
decision = 1

**Possui algum filtro?**

In [129]:
var_filter = ''

**Gostaria de renomear a variável?**

In [130]:
renomear = ''

**Registrando decisão**

In [131]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `def_loc_nasc`

Local de nascimento. Nominal, com as seguintes classificações:
- Hospital;
- Outros estabelecimentos de saúde;
- Domicílio;
- Via pública;
- Outros;
- Ignorado

In [132]:
var = 'def_loc_nasc'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [133]:
decision = 1

**Possui algum filtro?**

In [134]:
var_filter = ''

**Gostaria de renomear a variável?**

In [135]:
renomear = ''

**Registrando decisão**

In [136]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `LOCNASC`

Local de ocorrência do nascimento, conforme a tabela:
- 9: Ignorado;
- 1: Hospital;
- 2: Outro Estab Saúde;
- 3: Domicílio;
- 4: Outros

In [137]:
var = 'LOCNASC'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [138]:
decision = 3

**Possui algum filtro?**

In [139]:
var_filter = ''

**Gostaria de renomear a variável?**

In [140]:
renomear = ''

**Registrando decisão**

In [141]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `CODMUNRES`

Município de residência da mãe, em codificação idêntica a deCODMUNNASC, conforme tabela TABMUN

In [142]:
var = 'CODMUNRES'
plot_db_var(db_sinasc, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [143]:
decision = 1

**Possui algum filtro?**

In [144]:
var_filter = ''

**Gostaria de renomear a variável?**

In [145]:
renomear = ''

**Registrando decisão**

In [146]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `res_codigo_adotado`

Armazena o código atribuído ao município de residência atualmente, tratando os casos em que múltiplos códigos tenham sido utilizados para um mesmo município ao longo do tempo

In [147]:
var = 'res_codigo_adotado'
plot_db_var(db_sinasc, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [148]:
decision = 1

**Possui algum filtro?**

In [149]:
var_filter = ''

**Gostaria de renomear a variável?**

In [150]:
renomear = ''

**Registrando decisão**

In [151]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `res_MUNNOME`

Nome do município de residência

In [152]:
var = 'res_MUNNOME'
plot_db_var(db_sinasc, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [153]:
decision = 1

**Possui algum filtro?**

In [154]:
var_filter = ''

**Gostaria de renomear a variável?**

In [155]:
renomear = ''

**Registrando decisão**

In [156]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `res_MUNNOMEX`

Nome do município de residência em maiúsculas e sem acentos

In [157]:
var = 'res_MUNNOMEX'
plot_db_var(db_sinasc, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [158]:
decision = 1

**Possui algum filtro?**

In [159]:
var_filter = ''

**Gostaria de renomear a variável?**

In [160]:
renomear = ''

**Registrando decisão**

In [161]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `res_RSAUDCOD`

Código da Regional de Saúde a que o Município de residência pertence

In [162]:
var = 'res_RSAUDCOD'
plot_db_var(db_sinasc, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [163]:
decision = 1

**Possui algum filtro?**

In [164]:
var_filter = ''

**Gostaria de renomear a variável?**

In [165]:
renomear = ''

**Registrando decisão**

In [166]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

## Pai

### `IDADEPAI`

Idade do pai

In [167]:
var = 'IDADEPAI'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [168]:
decision = 3

**Possui algum filtro?**

In [169]:
var_filter = ''

**Gostaria de renomear a variável?**

In [170]:
renomear = ''

**Registrando decisão**

In [171]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

## Nascido

### `DTNASC`

Data do nascimento, no formato ddmmaaaa

In [172]:
var = 'DTNASC'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [173]:
decision = 1

**Possui algum filtro?**

In [174]:
var_filter = ''

**Gostaria de renomear a variável?**

In [175]:
renomear = ''

**Registrando decisão**

In [176]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `def_sexo`

Sexo. Nominal, com as seguintes classificações:
- Masculino;
- Feminino;
- Ignorado

In [177]:
var = 'def_sexo'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [178]:
decision = 1

**Possui algum filtro?**

In [179]:
var_filter = ''

**Gostaria de renomear a variável?**

In [180]:
renomear = ''

**Registrando decisão**

In [181]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `PESO`

Peso ao nascer, em gramas

In [182]:
var = 'PESO'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [183]:
decision = 1

**Possui algum filtro?**

In [184]:
var_filter = ''

**Gostaria de renomear a variável?**

In [185]:
renomear = ''

**Registrando decisão**

In [186]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `def_raca_cor`

Raça/cor. Nominal, com as seguintes classificações:
- Branca;
- Preta;
- Amarela;
- Parda;
- Indígena

In [187]:
var = 'def_raca_cor'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [188]:
decision = 1

**Possui algum filtro?**

In [189]:
var_filter = ''

**Gostaria de renomear a variável?**

In [190]:
renomear = ''

**Registrando decisão**

In [191]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `RACACOR`

Raça/Cor:
- 1: Branca;
- 2: Preta;
- 3: Amarela;
- 4: Parda;
- 5: Indígena

In [192]:
var = 'RACACOR'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [193]:
decision = 3

**Possui algum filtro?**

In [194]:
var_filter = ''

**Gostaria de renomear a variável?**

In [195]:
renomear = ''

**Registrando decisão**

In [196]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `def_parto_prematuro`

Indica a prematuridade do nascimento.
- Termo: não há indícios de prematuridade;
- Inconclusivo-IG: há indício de prematuridade dado pela idade gestacional (`GESTACAO<=4`);
- Inconclusivo-Peso: há indício de prematuridade dado pelo peso ao nascer (`PESO<2500`);
- Prematuro: a idade gestacional e o peso ao nascer indicam prematuridade

In [197]:
var = 'def_parto_prematuro'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [198]:
decision = 3

**Possui algum filtro?**

In [199]:
var_filter = ''

**Gostaria de renomear a variável?**

In [200]:
renomear = ''

**Registrando decisão**

In [201]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `parto_prematuro`

Indica a prematuridade do nascimento.

- 0: não há indícios de prematuridade;
- 1: há indício de prematuridade dado pela idade gestacional (`GESTACAO<=4`);
- 2: há indício de prematuridade dado pelo peso ao nascer (`PESO<2500`);
- 3: a idade gestacional e o peso ao nascer indicam prematuridade

In [202]:
var = 'parto_prematuro'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [203]:
decision = 3

**Possui algum filtro?**

In [204]:
var_filter = ''

**Gostaria de renomear a variável?**

In [205]:
renomear = ''

**Registrando decisão**

In [206]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `TPAPRESENT`

Tipo de apresentação do RN. Valores:
- 1 – Cefálico;
- 2 – Pélvica ou podálica;
- 3 – Transversa;
- 9 – Ignorado.

In [207]:
var = 'TPAPRESENT'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [208]:
decision = 3

**Possui algum filtro?**

In [209]:
var_filter = ''

**Gostaria de renomear a variável?**

In [210]:
renomear = ''

**Registrando decisão**

In [211]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `DIFDATA`

Diferença entre a data de óbito e data do recebimento original da DO ([`DTNASC`] – [`DTRECORIG`])

In [212]:
var = 'DIFDATA'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [213]:
decision = 3

**Possui algum filtro?**

In [214]:
var_filter = ''

**Gostaria de renomear a variável?**

In [215]:
renomear = ''

**Registrando decisão**

In [216]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

## Gestante


### `IDADEMAE`

Idade da mãe em anos

In [217]:
var = 'IDADEMAE'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [218]:
decision = 1

**Possui algum filtro?**

In [219]:
var_filter = ''

**Gostaria de renomear a variável?**

In [220]:
renomear = ''

**Registrando decisão**

In [221]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `def_parto`

Tipo de parto Nominal, com as seguintes classificações:
- Vaginal;
- Cesáreo;
- Ignorado

In [222]:
var = 'def_parto'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [223]:
decision = 2

**Possui algum filtro?**

In [224]:
var_filter = ''

**Gostaria de renomear a variável?**

In [225]:
renomear = ''

**Registrando decisão**

In [226]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `PARTO`

Tipo de parto, conforme a tabela:
- 9: Ignorado;
- 1: Vaginal;
- 2: Cesáreo

In [227]:
var = 'PARTO'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [228]:
decision = 3

**Possui algum filtro?**

In [229]:
var_filter = ''

**Gostaria de renomear a variável?**

In [230]:
renomear = ''

**Registrando decisão**

In [231]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `STCESPARTO`

Cesárea ocorreu antes do trabalho de parto iniciar? Valores:
- 1 – Sim;
- 2 – Não;
- 3 – Não se aplica;
- 9 – Ignorado.

In [232]:
var = 'STCESPARTO'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [233]:
decision = 3

**Possui algum filtro?**

In [234]:
var_filter = ''

**Gostaria de renomear a variável?**

In [235]:
renomear = ''

**Registrando decisão**

In [236]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `def_gravidez`

Tipo de gravidez Nominal, com as seguintes classificações:
- Única;
- Dupla;
- Tripla e mais;
- Ignorada

In [237]:
var = 'def_gravidez'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [238]:
decision = 3

**Possui algum filtro?**

In [239]:
var_filter = ''

**Gostaria de renomear a variável?**

In [240]:
renomear = ''

**Registrando decisão**

In [241]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `GRAVIDEZ`

Tipo de gravidez, conforme a tabela:
- 9: Ignorado;
- 1: Única;
- 2: Dupla;
- 3: Tripla e mais

In [242]:
var = 'GRAVIDEZ'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [243]:
decision = 1

**Possui algum filtro?**

In [244]:
var_filter = ''

**Gostaria de renomear a variável?**

In [245]:
renomear = ''

**Registrando decisão**

In [246]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `def_gestacao`

Semana de gestação Nominal, com as seguintes classificações:
- Menos de 22 semanas;
- 22 a 27 semanas;
- 28 a 31 semanas;
- 32 a 36 semanas;
- 37 a 41 semanas;
- 42 semanas ou mais

In [247]:
var = 'def_gestacao'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [248]:
decision = 1

**Possui algum filtro?**

In [249]:
var_filter = ''

**Gostaria de renomear a variável?**

In [250]:
renomear = ''

**Registrando decisão**

In [251]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `GESTACAO`

Semanas de gestação, conforme a tabela:
- 9: Ignorado;
- 1: Menos de 22 semanas;
- 2: 22 a 27 semanas;
- 3: 28 a 31 semanas;
- 4: 32 a 36 semanas;
- 5: 37 a 41 semanas;
- 6: 42 semanas e mais

In [252]:
var = 'GESTACAO'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [253]:
decision = 2

**Possui algum filtro?**

In [254]:
var_filter = ''

**Gostaria de renomear a variável?**

In [255]:
renomear = ''

**Registrando decisão**

In [256]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `SEMAGESTAC`

Número de semanas de gestação.

In [257]:
var = 'SEMAGESTAC'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [258]:
decision = 3

**Possui algum filtro?**

In [259]:
var_filter = ''

**Gostaria de renomear a variável?**

In [260]:
renomear = ''

**Registrando decisão**

In [261]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `def_est_civil`

Estado civil Situação conjugal:
- Solteiro;
- Casado;
- Viúvo;
- Separado judicialmente/divorciado;
- União estável;
- Ignorado

In [262]:
var = 'def_est_civil'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [263]:
decision = 1

**Possui algum filtro?**

In [264]:
var_filter = ''

**Gostaria de renomear a variável?**

In [265]:
renomear = ''

**Registrando decisão**

In [266]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `ESTCIVMAE`

Estado civil, conforme a tabela:
- 1: Solteira;
- 2: Casada;
- 3: Viuva;
- 4: Separado judicialmente/Divorciado;
- 5: União consensual (versões anteriores);
- 9: Ignorado

In [267]:
var = 'ESTCIVMAE'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [268]:
decision = 3

**Possui algum filtro?**

In [269]:
var_filter = ''

**Gostaria de renomear a variável?**

In [270]:
renomear = ''

**Registrando decisão**

In [271]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `def_escol_mae`

Escolaridade da mãe
- Nenhuma;
- de 1 a 3 anos;
- de 4 a 7 anos;
- 8 a 11 anos;
- 12 anos e mais;
- Ignorado

In [272]:
var = 'def_escol_mae'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [273]:
decision = 1

**Possui algum filtro?**

In [274]:
var_filter = ''

**Gostaria de renomear a variável?**

In [275]:
renomear = ''

**Registrando decisão**

In [276]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `ESCMAE`

Escolaridade, anos de estudo concluídos:
- 1: Nenhuma;
- 2: 1 a 3 anos;
- 3: 4 a 7 anos;
- 4: 8 a 11 anos;
- 5: 12 e mais;
- 9: Ignorado

In [277]:
var = 'ESCMAE'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [278]:
decision = 3

**Possui algum filtro?**

In [279]:
var_filter = ''

**Gostaria de renomear a variável?**

In [280]:
renomear = ''

**Registrando decisão**

In [281]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `ESCMAE2010`

Escolaridade 2010. Valores:
- 0 – Sem escolaridade;
- 1 – Fundamental I (1a a 4a série);
- 2 – Fundamental II (5a a 8a série);
- 3 – Médio (antigo 2o Grau);
- 4 – Superior incompleto;
- 5 – Superior completo;
- 9 – Ignorado.

In [282]:
var = 'ESCMAE2010'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [283]:
decision = 3

**Possui algum filtro?**

In [284]:
var_filter = ''

**Gostaria de renomear a variável?**

In [285]:
renomear = ''

**Registrando decisão**

In [286]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `ESCMAEAGR1`

Escolaridade 2010 agregada. Valores:
- 00 – Sem Escolaridade;
- 01 – Fundamental I Incompleto;
- 02 – Fundamental I Completo;
- 03 – Fundamental II Incompleto;
- 04 – Fundamental II Completo;
- 05 – Ensino Médio Incompleto;
- 06 – Ensino Médio Completo;
- 07 – Superior Incompleto;
- 08 – Superior Completo;
- 09 – Ignorado;
- 10 – Fundamental I Incompleto ou Inespecífico;
- 11 – Fundamental II Incompleto ou Inespecífico;
- 12 – EnsinoMédio Incompleto ou Inespecífico.

In [287]:
var = 'ESCMAEAGR1'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [288]:
decision = 3

**Possui algum filtro?**

In [289]:
var_filter = ''

**Gostaria de renomear a variável?**

In [290]:
renomear = ''

**Registrando decisão**

In [291]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `SERIESCMAE`

Série escolar da mãe. Valores de 1 a 8.

In [292]:
var = 'SERIESCMAE'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [293]:
decision = 3

**Possui algum filtro?**

In [294]:
var_filter = ''

**Gostaria de renomear a variável?**

In [295]:
renomear = ''

**Registrando decisão**

In [296]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `CODOCUPMAE`

Ocupação, conforme a Classificação Brasileira de Ocupações (CBO-2002)

In [297]:
var = 'CODOCUPMAE'
plot_db_var(db_sinasc, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [298]:
decision = 2

**Possui algum filtro?**

In [299]:
var_filter = ''

**Gostaria de renomear a variável?**

In [300]:
renomear = ''

**Registrando decisão**

In [301]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `RACACORMAE`

Raça/cor da mãe

- 1: Branca;
- 2: Preta;
- 3: Amarela;
- 4: Parda;
- 5: Indígena

In [302]:
var = 'RACACORMAE'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [303]:
decision = 1

**Possui algum filtro?**

In [304]:
var_filter = ''

**Gostaria de renomear a variável?**

In [305]:
renomear = ''

**Registrando decisão**

In [306]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `QTDFILVIVO`

Número de filhos vivos

In [307]:
var = 'QTDFILVIVO'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [308]:
decision = 3

**Possui algum filtro?**

In [309]:
var_filter = ''

**Gostaria de renomear a variável?**

In [310]:
renomear = ''

**Registrando decisão**

In [311]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `QTDFILMORT`

Número de filhos mortos

In [312]:
var = 'QTDFILMORT'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [313]:
decision = 3

**Possui algum filtro?**

In [314]:
var_filter = ''

**Gostaria de renomear a variável?**

In [315]:
renomear = ''

**Registrando decisão**

In [316]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `QTDGESTANT`

Número de gestações anteriores

In [317]:
var = 'QTDGESTANT'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [318]:
decision = 3

**Possui algum filtro?**

In [319]:
var_filter = ''

**Gostaria de renomear a variável?**

In [320]:
renomear = ''

**Registrando decisão**

In [321]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `QTDPARTNOR`

Número de partos vaginais

In [322]:
var = 'QTDPARTNOR'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [323]:
decision = 3

**Possui algum filtro?**

In [324]:
var_filter = ''

**Gostaria de renomear a variável?**

In [325]:
renomear = ''

**Registrando decisão**

In [326]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### `QTDPARTCES`

Número de partos cesáreos

In [327]:
var = 'QTDPARTCES'
plot_db_var(db_sinasc, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [328]:
decision = 3

**Possui algum filtro?**

In [329]:
var_filter = ''

**Gostaria de renomear a variável?**

In [330]:
renomear = ''

**Registrando decisão**

In [331]:
vars_sinasc.append({
  'Database' : db_sinasc,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

## RESUMO VARIÁVEIS SINASC

### Tabela resultante

In [332]:
decisao_vars_sinasc = pd.DataFrame(vars_sinasc)
decisao_vars_sinasc

,Database,Variável,Status,Filter,Renomear
0,sinasc,CODESTAB,1,,cnes_hosp
1,sinasc,STDNNOVA,3,,
2,sinasc,STDNEPIDEM,3,,
3,sinasc,ano_nasc,1,ano_nasc between 2010 and 2019,ano
4,sinasc,APGAR1,2,APGAR1 between 0 and 10,
5,sinasc,APGAR5,2,APGAR5 between 0 and 10,
6,sinasc,def_consultas,2,"def_consultas != ""Ignorado""",pre_natal
7,sinasc,CONSULTAS,3,,
8,sinasc,CONSPRENAT,3,,
9,sinasc,STTRABPART,3,,


# Variáveis do SIH

In [333]:
db_sih = 'sih'
vars_sih = list()

### Hospital

#### `CNES`

Código CNES do hospital

In [334]:
var = 'CNES'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [335]:
decision = 1

**Possui algum filtro?**

In [336]:
var_filter = ''

**Gostaria de renomear a variável?**

In [337]:
renomear = ''

**Registrando decisão**

In [338]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### Procedimento

#### ano_internacao

Ano de internação

In [339]:
var = 'ano_internacao'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [340]:
decision = 1

**Possui algum filtro?**

In [341]:
var_filter = ''

**Gostaria de renomear a variável?**

In [342]:
renomear = ''

**Registrando decisão**

In [343]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `ANO_CMPT`

Ano de processamento da AIH

In [344]:
var = 'ANO_CMPT'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [345]:
decision = 3

**Possui algum filtro?**

In [346]:
var_filter = ''

**Gostaria de renomear a variável?**

In [347]:
renomear = ''

**Registrando decisão**

In [348]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_procedimento_realizado`

Definição do procedimento realizado

In [349]:
var = 'def_procedimento_realizado'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [350]:
decision = 3

**Possui algum filtro?**

In [351]:
var_filter = ''

**Gostaria de renomear a variável?**

In [352]:
renomear = ''

**Registrando decisão**

In [353]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `PROC_REA`

Procedimento realizado

In [354]:
var = 'PROC_REA'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [355]:
decision = 1

**Possui algum filtro?**

In [356]:
var_filter = ''

**Gostaria de renomear a variável?**

In [357]:
renomear = ''

**Registrando decisão**

In [358]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_car_int`

Definição do caráter da internação

In [359]:
var = 'def_car_int'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [360]:
decision = 3

**Possui algum filtro?**

In [361]:
var_filter = ''

**Gostaria de renomear a variável?**

In [362]:
renomear = ''

**Registrando decisão**

In [363]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### Localidade

#### `MUNIC_MOV`

Município do estabelecimento

In [364]:
var = 'MUNIC_MOV'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [365]:
decision = 3

**Possui algum filtro?**

In [366]:
var_filter = ''

**Gostaria de renomear a variável?**

In [367]:
renomear = ''

**Registrando decisão**

In [368]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `int_MUNCOD`

Código do município do estabelecimento de internação

In [369]:
var = 'int_MUNCOD'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [370]:
decision = 3

**Possui algum filtro?**

In [371]:
var_filter = ''

**Gostaria de renomear a variável?**

In [372]:
renomear = ''

**Registrando decisão**

In [373]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `int_codigo_adotado`

Armazena o código atribuído ao município do estabelecimento de internação, tratando os casos em que múltiplos códigos tenham sido utilizados para um mesmo município ao longo do tempo

In [374]:
var = 'int_codigo_adotado'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [375]:
decision = 1

**Possui algum filtro?**

In [376]:
var_filter = ''

**Gostaria de renomear a variável?**

In [377]:
renomear = ''

**Registrando decisão**

In [378]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `int_MUNNOME`

Nome (acentuado, maiúsculas e minúsculas) do Município (padrão DOS, página de código 850) do estabelecimento de internação

In [379]:
var = 'int_MUNNOME'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [380]:
decision = 1

**Possui algum filtro?**

In [381]:
var_filter = ''

**Gostaria de renomear a variável?**

In [382]:
renomear = ''

**Registrando decisão**

In [383]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `int_MUNNOMEX`

Nome (sem acentos, em maiúsculas) do Município do estabelecimento de internação

In [384]:
var = 'int_MUNNOMEX'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [385]:
decision = 1

**Possui algum filtro?**

In [386]:
var_filter = ''

**Gostaria de renomear a variável?**

In [387]:
renomear = ''

**Registrando decisão**

In [388]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_reg_metr_int`

Região metropolitana do estabelecimento de internação

In [389]:
var = 'def_reg_metr_int'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [390]:
decision = 3

**Possui algum filtro?**

In [391]:
var_filter = ''

**Gostaria de renomear a variável?**

In [392]:
renomear = ''

**Registrando decisão**

In [393]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_rsaud_int`

Região de saúde do estabelecimento de internação

In [394]:
var = 'def_rsaud_int'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [395]:
decision = 3

**Possui algum filtro?**

In [396]:
var_filter = ''

**Gostaria de renomear a variável?**

In [397]:
renomear = ''

**Registrando decisão**

In [398]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `int_RSAUDCOD`

Código da Regional de Saúde a que o Município do estabelecimento de internação

In [399]:
var = 'int_RSAUDCOD'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [400]:
decision = 1

**Possui algum filtro?**

In [401]:
var_filter = ''

**Gostaria de renomear a variável?**

In [402]:
renomear = ''

**Registrando decisão**

In [403]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `MUNIC_RES`

Município de residência do paciente

In [404]:
var = 'MUNIC_RES'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [405]:
decision = 3

**Possui algum filtro?**

In [406]:
var_filter = ''

**Gostaria de renomear a variável?**

In [407]:
renomear = ''

**Registrando decisão**

In [408]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `res_MUNCOD`

Código do município de residência do paciente

In [409]:
var = 'res_MUNCOD'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [410]:
decision = 3

**Possui algum filtro?**

In [411]:
var_filter = ''

**Gostaria de renomear a variável?**

In [412]:
renomear = ''

**Registrando decisão**

In [413]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `res_codigo_adotado`

Armazena o código atribuído ao município de residência do paciente, tratando os casos em que múltiplos códigos tenham sido utilizados para um mesmo município ao longo do tempo

In [414]:
var = 'res_codigo_adotado'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [415]:
decision = 1

**Possui algum filtro?**

In [416]:
var_filter = ''

**Gostaria de renomear a variável?**

In [417]:
renomear = ''

**Registrando decisão**

In [418]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `res_MUNNOME`

Nome (acentuado, maiúsculas e minúsculas) do Município (padrão DOS, página de código 850) de residência do paciente

In [419]:
var = 'res_MUNNOME'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [420]:
decision = 1

**Possui algum filtro?**

In [421]:
var_filter = ''

**Gostaria de renomear a variável?**

In [422]:
renomear = ''

**Registrando decisão**

In [423]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `res_MUNNOMEX`

Nome (sem acentos, em maiúsculas) do Município de residência do paciente

In [424]:
var = 'res_MUNNOMEX'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [425]:
decision = 1

**Possui algum filtro?**

In [426]:
var_filter = ''

**Gostaria de renomear a variável?**

In [427]:
renomear = ''

**Registrando decisão**

In [428]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_reg_metr_res`

Região metropolitana de residência do paciente

In [429]:
var = 'def_reg_metr_res'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [430]:
decision = 3

**Possui algum filtro?**

In [431]:
var_filter = ''

**Gostaria de renomear a variável?**

In [432]:
renomear = ''

**Registrando decisão**

In [433]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_rsaud_res`

Região de saúde de residência do paciente

In [434]:
var = 'def_rsaud_res'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [435]:
decision = 3

**Possui algum filtro?**

In [436]:
var_filter = ''

**Gostaria de renomear a variável?**

In [437]:
renomear = ''

**Registrando decisão**

In [438]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `res_RSAUDCOD`

Código da Regional de Saúde a que o Município de residência do paciente pertence

In [439]:
var = 'res_RSAUDCOD'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [440]:
decision = 1

**Possui algum filtro?**

In [441]:
var_filter = ''

**Gostaria de renomear a variável?**

In [442]:
renomear = ''

**Registrando decisão**

In [443]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

### Gestante

#### `IDADE`

Idade

In [444]:
var = 'IDADE'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [445]:
decision = 3

**Possui algum filtro?**

In [446]:
var_filter = ''

**Gostaria de renomear a variável?**

In [447]:
renomear = ''

**Registrando decisão**

In [448]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_idade_anos`

Idade em anos

In [449]:
var = 'def_idade_anos'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [450]:
decision = 2

**Possui algum filtro?**

In [451]:
var_filter = ''

**Gostaria de renomear a variável?**

In [452]:
renomear = ''

**Registrando decisão**

In [453]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_idade_meses`

Idade em meses

In [454]:
var = 'def_idade_meses'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [455]:
decision = 3

**Possui algum filtro?**

In [456]:
var_filter = ''

**Gostaria de renomear a variável?**

In [457]:
renomear = ''

**Registrando decisão**

In [458]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_idade_dias`

Idade em dias

In [459]:
var = 'def_idade_dias'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [460]:
decision = 3

**Possui algum filtro?**

In [461]:
var_filter = ''

**Gostaria de renomear a variável?**

In [462]:
renomear = ''

**Registrando decisão**

In [463]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_idade_bas`

Definição de faixa etária 1

In [464]:
var = 'def_idade_bas'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [465]:
decision = 3

**Possui algum filtro?**

In [466]:
var_filter = ''

**Gostaria de renomear a variável?**

In [467]:
renomear = ''

**Registrando decisão**

In [468]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_idade_pub`

Definição de faixa etária 2

In [469]:
var = 'def_idade_pub'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [470]:
decision = 3

**Possui algum filtro?**

In [471]:
var_filter = ''

**Gostaria de renomear a variável?**

In [472]:
renomear = ''

**Registrando decisão**

In [473]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_idade_18`

Definição de faixa etária 3

In [474]:
var = 'def_idade_18'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [475]:
decision = 3

**Possui algum filtro?**

In [476]:
var_filter = ''

**Gostaria de renomear a variável?**

In [477]:
renomear = ''

**Registrando decisão**

In [478]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_raca_cor`

Definição de raça/cor do paciente

In [479]:
var = 'def_raca_cor'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [480]:
decision = 3

**Possui algum filtro?**

In [481]:
var_filter = ''

**Gostaria de renomear a variável?**

In [482]:
renomear = ''

**Registrando decisão**

In [483]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `RACA_COR`

Raça/cor do paciente

In [484]:
var = 'RACA_COR'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [485]:
decision = 3

**Possui algum filtro?**

In [486]:
var_filter = ''

**Gostaria de renomear a variável?**

In [487]:
renomear = ''

**Registrando decisão**

In [488]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_cbo`

Definição da Classificação Brasileira de Ocupações (CBO)

In [489]:
var = 'def_cbo'
plot_db_var(db_sih, var, True).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [490]:
decision = 3

**Possui algum filtro?**

In [491]:
var_filter = ''

**Gostaria de renomear a variável?**

In [492]:
renomear = ''

**Registrando decisão**

In [493]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `CBOR`

Ocupação do paciente, segundo a Classificação Brasileira de Ocupações (CBO)

In [494]:
var = 'CBOR'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [495]:
decision = 3

**Possui algum filtro?**

In [496]:
var_filter = ''

**Gostaria de renomear a variável?**

In [497]:
renomear = ''

**Registrando decisão**

In [498]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_instru`

Definição do grau de instrução do paciente

In [499]:
var = 'def_instru'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [500]:
decision = 3

**Possui algum filtro?**

In [501]:
var_filter = ''

**Gostaria de renomear a variável?**

In [502]:
renomear = ''

**Registrando decisão**

In [503]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `INSTRU`

Grau de instrução do paciente

In [504]:
var = 'INSTRU'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [505]:
decision = 3

**Possui algum filtro?**

In [506]:
var_filter = ''

**Gostaria de renomear a variável?**

In [507]:
renomear = ''

**Registrando decisão**

In [508]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_num_filhos`

Definição do número de filhos do paciente

In [509]:
var = 'def_num_filhos'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [510]:
decision = 3

**Possui algum filtro?**

In [511]:
var_filter = ''

**Gostaria de renomear a variável?**

In [512]:
renomear = ''

**Registrando decisão**

In [513]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `NUM_FILHOS`

Número de filhos do paciente

In [514]:
var = 'NUM_FILHOS'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [515]:
decision = 3

**Possui algum filtro?**

In [516]:
var_filter = ''

**Gostaria de renomear a variável?**

In [517]:
renomear = ''

**Registrando decisão**

In [518]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `GESTRISCO`

Indicador se é gestante de risco

In [519]:
var = 'GESTRISCO'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [520]:
decision = 3

**Possui algum filtro?**

In [521]:
var_filter = ''

**Gostaria de renomear a variável?**

In [522]:
renomear = ''

**Registrando decisão**

In [523]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `def_morte`

Definição de óbito

In [524]:
var = 'def_morte'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [525]:
decision = 3

**Possui algum filtro?**

In [526]:
var_filter = ''

**Gostaria de renomear a variável?**

In [527]:
renomear = ''

**Registrando decisão**

In [528]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

#### `MORTE`

Indica óbito

In [529]:
var = 'MORTE'
plot_db_var(db_sih, var).show()

**O que fazer com essa variável?**

- **1)** Usar, sem nenhum filtro
- **2)** Usar, mas com filtro
- **3)** Descartar

In [530]:
decision = 3

**Possui algum filtro?**

In [531]:
var_filter = ''

**Gostaria de renomear a variável?**

In [532]:
renomear = ''

**Registrando decisão**

In [533]:
vars_sih.append({
  'Database' : db_sih,
  'Variável' : var,
  'Status' : decision,
  'Filter' : var_filter,
  'Renomear' : renomear,
})

## RESUMO VARIÁVEIS SIH

### Tabela resultante

In [534]:
decisao_vars_sih = pd.DataFrame(vars_sih)
decisao_vars_sih

,Database,Variável,Status,Filter,Renomear
0,sih,CNES,1,,
1,sih,ano_internacao,1,,
2,sih,ANO_CMPT,3,,
3,sih,def_procedimento_realizado,3,,
4,sih,PROC_REA,1,,
5,sih,def_car_int,3,,
6,sih,MUNIC_MOV,3,,
7,sih,int_MUNCOD,3,,
8,sih,int_codigo_adotado,1,,
9,sih,int_MUNNOME,1,,
